In [2]:
import pandas as pd
import xlwings as xw
import openpyxl as op
from openpyxl import load_workbook
from datetime import datetime
from datetime import timedelta

# 判断给定日期是周几，返回0-6
def isweek(date):
    date_f = datetime.strptime(date, '%Y%m%d') # 字符串转为日期格式
    week = datetime.weekday(date_f)
    return(week)

# 复制单元格数据
def copy2cell(file_name, wb, ws_src, ws_dsc, src, dsc):
    ws_dsc[dsc].value = src  # 复制单元格的值
    # print('      ', ws_src, ws_dsc, '=====>', ws_dsc[dsc].value)
    wb.save(file_name)

# 写入单元格数据
def create2cell(file_name, wb, ws_src, ws_dsc, src, dsc):
    ws_dsc[dsc].value = src  # 写入单元格的值
    wb.save(file_name)


def create_fridy(file_name, src_sheet, dsc_sheet, dsc_list, beg, end):
    wb = load_workbook(file_name)
    ws_src = wb[src_sheet]
    ws_dsc = wb[dsc_sheet]
    date_list = [] 
    beg = datetime.strptime(beg, "%Y%m%d") 
    end = datetime.strptime(end, "%Y%m%d")
    
    while end >= beg: 
        date_str = end.strftime("%Y%m%d") 
        date_list.append(date_str) 
        end -= timedelta(days=1) 
    # print(date_list)
    s = 2
    for i in date_list:
        i = str(i.replace('-',''))
        if isweek(i) == 4:
            dsc = dsc_list + str(s)
            i = int(i)
            create2cell(file_name, wb, ws_src, ws_dsc, i, dsc)
            s = s+1

# 将每周六的收盘收据汇总
def move_dates(file_name, src_sheet, dsc_sheet, dsc_list):
    wb = load_workbook(file_name)
    ws_src = wb[src_sheet]
    ws_dsc = wb[dsc_sheet]
    m = 2
    n = 2 
    for row in ws_src.iter_rows():
        row[2].value = str(row[2].value.replace('-',''))
        if row[2].value != '日期' and isweek(row[2].value) == 4:
            if dsc_list == 'B' or dsc_list == 'G':
                dsc = dsc_list + str(m)
                copy2cell(file_name, wb, ws_src, ws_dsc, row[2].value, dsc)
                m = m +1
            else:
                dsc = dsc_list + str(n)
                copy2cell(file_name, wb, ws_src, ws_dsc, row[4].value, dsc)
                n = n +1
            

beg = '20230601'  # 开始日期
end = '20230913'  # 结束日期
# end = datetime.today().strftime("%Y%m%d")  # 取当前日期转换为20230822格式
file = r'E:\大徐\99 Python\20_Invest\stock' + '\\' + end + '.xlsx'

create_fridy(file, '8 - 春秋航空', '汇总', 'B', beg, end)

# move_dates(file, '8 - 春秋航空', '汇总', 'B')
move_dates(file, '8 - 春秋航空', '汇总', 'C')
move_dates(file, '9 - 洋河股份', '汇总', 'D')
move_dates(file, '10 - 分众传媒', '汇总', 'E')

# move_dates(file, '11 - 腾讯控股', '汇总', 'G')
move_dates(file, '11 - 腾讯控股', '汇总', 'H')
move_dates(file, '12 - 小米集团', '汇总', 'I')
move_dates(file, '13 - 福寿园', '汇总', 'J')
print('Finish !')

Finish !


In [7]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
# print('login respond error_code:'+lg.error_code)
# print('login respond  error_msg:'+lg.error_msg)

# 获取指数(综合指数、规模指数、一级行业指数、二级行业指数、策略指数、成长指数、价值指数、主题指数)K线数据
# 综合指数，例如：sh.000001 上证指数，sz.399106 深证综指 等；
# 规模指数，例如：sh.000016 上证50，sh.000300 沪深300，sh.000905 中证500，sz.399001 深证成指等；
# 一级行业指数，例如：sh.000037 上证医药，sz.399433 国证交运 等；
# 二级行业指数，例如：sh.000952 300地产，sz.399951 300银行 等；
# 策略指数，例如：sh.000050 50等权，sh.000982 500等权 等；
# 成长指数，例如：sz.399376 小盘成长 等；
# 价值指数，例如：sh.000029 180价值 等；
# 主题指数，例如：sh.000015 红利指数，sh.000063 上证周期 等；
# 详细指标参数，参见“历史行情指标参数”章节；“周月线”参数与“日线”参数不同。
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg

rs = bs.query_history_k_data_plus(
    "sh.000300",
    "date,open,close",
    start_date='2017-01-01',
    end_date='2017-06-30',
    frequency="d")

print(rs)
# print('query_history_k_data_plus respond error_code:'+rs.error_code)
# print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

# 打印结果集
# data_list = []
# while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    # data_list.append(rs.get_row_data())
# result = pd.DataFrame(data_list, columns=rs.fields)
# 结果集输出到csv文件
# result.to_csv(r'E:\大徐\99 Python\20_Invest\stock\123.csv', index=False)

# print(result)

# 登出系统
bs.logout()

login success!
logout success!


In [ ]:
import datetime
def get_fridays(year):
fridays = []

# 构造日期范围，从1月1日到12月31日
start_date = datetime.date(year, 1, 1)
end_date = datetime.date(year, 12, 31)

# 构造一个日期循环器
d = start_date
delta = datetime.timedelta(days=1)
while d<= end_date:

# 如果当前日期是周五，就将其加入到列表中
if d.weekday() == 4:
fridays.append(d)
d += delta
return fridays
if __name__ == '__main__':
year = int(input('请输入年份：'))
fridays = get_fridays(year)
print(f'{year}年总共有{len(fridays)}个周五：')
for friday in fridays:
print(friday)